# NOIRE-Net-analyze: This notebook loads the trained NOIRE-Net models and automatically classifies and scales ionograms

## 1 - Import libaries 

In [1]:
from tensorflow.keras.models import load_model
import numpy as np
import os
from PIL import Image, ImageDraw
from statistics import median
import os
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

## 2 - Load NOIRE-Net models

In [2]:
# Load E-classify models 
e_classify_models = [load_model(f'/Users/akv020/Tensorflow/NOIRE-Net/NOIRE-Net/E-classify/E-classify_run{i}.h5') for i in range(1, 11)]

# Load E-scale models
e_scale_models = [load_model(f'/Users/akv020/Tensorflow/NOIRE-Net/NOIRE-Net/E-scale/E-scale_run{i}.h5') for i in range(1, 11)]

# Load F-classify models
f_classify_models = [load_model(f'/Users/akv020/Tensorflow/NOIRE-Net/NOIRE-Net/F-classify/F-classify_run{i}.h5') for i in range(1, 11)]

# Load F-scale models
f_scale_models = [load_model(f'/Users/akv020/Tensorflow/NOIRE-Net/NOIRE-Net/F-scale/F-scale_run{i}.h5') for i in range(1, 11)]

Metal device set to: Apple M1 Max


2024-03-20 11:33:21.900621: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-20 11:33:21.900800: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## 3 - Define directory containing black-and-white ionograms in .png format with dimension (310x310)

In [3]:
# Directory where the images are stored
image_dir = '/Users/akv020/Tensorflow/NOIRE-Net/data/2024-03-19/ionograms/'
result_dir = '/Users/akv020/Tensorflow/NOIRE-Net/data/2024-03-19/NOIRE-Net-analyzed/'


## 4 - Define a function to classify and scale ionograms using the NOIRE-Net models

In [4]:
# This code classify a PNG image in a specified directory using two sets of CNN models
# (E-classify and F-classify). Positive ionograms are then processed by the CNN regression models:
# (E-scale and F-scale), which computes the median and standard deviation of their regression outputs,
# and stores these results along with their standard deviations in respective dictionaries for the
# E-region and F-region analyses.

# Function to both classify and scale (if positive class) an ionogram
def classify_and_scale(image_array, classify_models, scale_models):
    
    # Use classify models to classify the ionogram into positive or negative class
    classifications = [model.predict(image_array, verbose=0)[0][0] for model in classify_models]
    positive_votes = sum(c > 0.5 for c in classifications)
    negative_votes = len(classify_models) - positive_votes

    # Handle tiebreaker with average accuracy
    if positive_votes == negative_votes:
        average_accuracy = np.mean(classifications)
        is_positive = average_accuracy > 0.5
    else:
        is_positive = positive_votes > negative_votes
    
    # For positive ionograms (containing an E or an F-region trace) scale the ionogram
    if is_positive:
        scale_results = [model.predict(image_array, verbose=0) for model in scale_models]
        median_result = np.median(scale_results, axis=0).tolist()
        std_result = np.std(scale_results, axis=0).tolist()
        
    # For negative ionograms, populate dictonary index with "nans" 
    else:
        median_result, std_result = [np.nan, np.nan], [np.nan, np.nan]
    
    # Return the results
    return median_result, std_result

## 5 - Iterate over all images in a directory containing the input ionograms (.png files)

In [5]:
# Initialize dictionaries to store results and standard deviations for E and F regions
e_results, f_results = {}, {}
e_std_devs, f_std_devs = {}, {}

# Process each image in the specified directory
#for image_name in os.listdir(image_dir):
#    if image_name.endswith('.png'):
        
for i, image_name in enumerate(os.listdir(image_dir)):
    if image_name.endswith('.png'):
        # Construct the full path to the image
        image_path = os.path.join(image_dir, image_name)

        # Read and preprocess the image
        # Convert to grayscale, resize, and normalize the pixel values
        img = Image.open(image_path).convert('L').resize((310, 310))
        img_array = np.expand_dims(np.array(img).astype('float32') / 255.0, axis=[0, -1])

        # Process the image with E-classify/E-scale models
        # Store the results and standard deviations for E-region
        e_results[image_name], e_std_devs[image_name] = classify_and_scale(img_array, e_classify_models, e_scale_models)

        # Process the image with F-classify/F-scale models
        # Store the results and standard deviations for F-region
        f_results[image_name], f_std_devs[image_name] = classify_and_scale(img_array, f_classify_models, f_scale_models)


2024-03-20 11:35:14.829879: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-03-20 11:35:14.955447: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-03-20 11:35:17.582374: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-03-20 11:35:17.855844: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-03-20 11:35:18.145526: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2024-03-20 11:35:18.427007: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-03-20 11:35:18.713194: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2024-03-20 11:35:19.013809: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-03-20 11:35:19.327186: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-03-20 11:35:19.665512: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-03-20 11:35:20.025736: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-03-20 11:35:20.315064: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-03-20 11:35:20.666000: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-03-20 11:35:20.995408: I tensorflow/core/grappler/optimizers/cust

## 6 - Define a function to draw the E-scale and F-scale lines on top of the ionogram

In [6]:
def draw_colored_lines_on_image(image_np, horizontal, vertical, std_horizontal, std_vertical, color='r'):
    # Convert the NumPy array to a PIL Image and ensure it's in RGB mode
    image = Image.fromarray(np.uint8(image_np))
    if image.mode != 'RGB':
        image = image.convert('RGB')

    # Create a draw object
    draw = ImageDraw.Draw(image)

    # Define the line color as an RGB tuple
    line_color = (255, 0, 0) if color == 'r' else (0, 0, 255)  # Red or blue

    # Draw horizontal and vertical lines if they are not NaN
    if not np.isnan(horizontal):
        draw.line([(0, horizontal), (image.width, horizontal)], fill=line_color, width=1)

    if not np.isnan(vertical):
        draw.line([(vertical, 0), (vertical, image.height)], fill=line_color, width=1)

    # Draw standard deviation lines if they are not NaN
    if not np.isnan(std_horizontal):
        upper_horizontal = max(0, min(image.height, horizontal + std_horizontal))
        lower_horizontal = max(0, min(image.height, horizontal - std_horizontal))
        draw.line([(0, upper_horizontal), (image.width, upper_horizontal)], fill=line_color, width=1)
        draw.line([(0, lower_horizontal), (image.width, lower_horizontal)], fill=line_color, width=1)

    if not np.isnan(std_vertical):
        upper_vertical = max(0, min(image.width, vertical + std_vertical))
        lower_vertical = max(0, min(image.width, vertical - std_vertical))
        draw.line([(upper_vertical, 0), (upper_vertical, image.height)], fill=line_color, width=1)
        draw.line([(lower_vertical, 0), (lower_vertical, image.height)], fill=line_color, width=1)

    return np.array(image)

## 7 - Define a function to acess regression results

In [7]:
# The function unpack_results checks if the input result is a nested list and, if so, unpacks
# and returns the inner list; otherwise, it returns the input as is.
def unpack_results(result):
    if isinstance(result, list) and len(result) == 1 and isinstance(result[0], list):
        return result[0]  # Unpack nested list
    return result  # Use as is

## 8 - Define functions to convert pixel values to a cusom scale and vise-verca

In [8]:
# This function converts a pixel value into a corresponding value on a custom scale. 
# Used to convert coordinates from an image to frequency in MHz or distance in km. 
# The function linearly interpolates the pixel value based on the range of the scale.
def convert_pixel_to_custom_scale(pixel, image_size, min_val, max_val):
    """
    Converts a pixel value to a value on a custom scale.
    
    Args:
    pixel (float): The pixel value to convert.
    image_size (int): The size of the image (in pixels) along the axis of interest.
    min_val (float): The minimum value of the custom scale.
    max_val (float): The maximum value of the custom scale.
    
    Returns:
    float: The corresponding value on the custom scale.
    """
    # Normalize the pixel value to a range [0, 1]
    normalized_pixel = pixel / image_size

    # Convert the normalized pixel to the custom scale
    return min_val + (max_val - min_val) * normalized_pixel

# Function to map custom values to pixel positions
def map_to_pixel(value, start, stop, num_pixels):
    return int((value - start) / (stop - start) * num_pixels)

## 9 - Save ionograms contianinng the scaling results for positive classes

In [9]:
# This code processes and saves images from a specified directory, overlaying them with colored lines
# representing E-region and F-region scaling results, and annotates each image with custom frequency
# and range scales, adjusting the title based on the presence of E and/or F-region features.

# Define the scale for frequency and virtual distance
range_start = 200058.33938693197 / 1e3  # km
range_stop = 1498436.9620081205 / 1e3  # km
freq_start = 0.549734235380000  # MHz
freq_stop = 15.992268665599999  # MHz

# Create frequency and range arrays
frequencies = np.linspace(freq_start, freq_stop, num=310)
ranges = np.linspace(range_start, range_stop, num=310)

# Custom frequency and range values for ticks
custom_frequencies = np.array([1, 3, 5, 7, 9, 11, 13, 15])  # MHz
custom_ranges = np.array([300, 500, 700, 900, 1100, 1300, 1500])  # km

# Process and save images with scaling results
for image_name in os.listdir(image_dir):
    if image_name.endswith('.png'):
        image_path = os.path.join(image_dir, image_name)
        result_path = os.path.join(result_dir, image_name)

        # Read and preprocess the image
        img = Image.open(image_path).convert('L')
        img_array = np.flipud(np.array(img))  # Convert to numpy array and flip vertically

        # Unpack results and standard deviations
        e_vertical, e_horizontal = unpack_results(e_results.get(image_name, [np.nan, np.nan]))
        f_vertical, f_horizontal = unpack_results(f_results.get(image_name, [np.nan, np.nan]))
        e_std_vertical, e_std_horizontal = unpack_results(e_std_devs.get(image_name, [np.nan, np.nan]))
        f_std_vertical, f_std_horizontal = unpack_results(f_std_devs.get(image_name, [np.nan, np.nan]))

        # Adjust coordinates for flipped image
        e_horizontal = img_array.shape[0] - e_horizontal
        f_horizontal = img_array.shape[0] - f_horizontal

        # Convert pixel values to frequency and range
        e_frequency = convert_pixel_to_custom_scale(e_vertical, img_array.shape[1], freq_start, freq_stop)
        f_frequency = convert_pixel_to_custom_scale(f_vertical, img_array.shape[1], freq_start, freq_stop)
        e_range = convert_pixel_to_custom_scale(e_horizontal, img_array.shape[0], range_start, range_stop)
        f_range = convert_pixel_to_custom_scale(f_horizontal, img_array.shape[0], range_start, range_stop)

        # Draw lines on the image
        img_with_e_lines = draw_colored_lines_on_image(img_array, e_horizontal, e_vertical, e_std_horizontal, e_std_vertical, color='r')
        img_with_f_lines = draw_colored_lines_on_image(img_array, f_horizontal, f_vertical, f_std_horizontal, f_std_vertical, color='g')

        # Combine images with lines
        combined_img = np.maximum(img_with_e_lines, img_with_f_lines)

        # Create a figure and axis for the plot
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.imshow(combined_img, cmap='gray', aspect='auto')

        # Set custom x-ticks and y-ticks
        x_tick_positions = [map_to_pixel(f, freq_start, freq_stop, combined_img.shape[1]) for f in custom_frequencies]
        y_tick_positions = [map_to_pixel(r, range_stop, range_start, combined_img.shape[0]) for r in custom_ranges]

        ax.set_xticks(x_tick_positions)
        ax.set_xticklabels([f"{f}" for f in custom_frequencies])
        ax.set_yticks(y_tick_positions)
        ax.set_yticklabels([f"{r}" for r in custom_ranges])

        ax.set_xlabel('Frequency [MHz]')
        ax.set_ylabel('Virtual distance [km]')

        # Set the title based on E and F-region results
        title = 'No trace'
        if not np.isnan(e_horizontal) and not np.isnan(f_horizontal):
            title = 'E and F-region'
        elif not np.isnan(e_horizontal):
            title = 'E-region'
        elif not np.isnan(f_horizontal):
            title = 'F-region'
        ax.set_title(title)

        # Adjust layout and save the image
        plt.tight_layout()  # Adjust layout
        plt.savefig(result_path, bbox_inches='tight')  # Save with reduced whitespace
        plt.close()

print("Processed images saved in the 'results' directory")

Processed images saved in the 'results' directory
